In [22]:
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import json

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

import pymysql
pymysql.install_as_MySQLdb()

from bs4 import BeautifulSoup
import requests
import pymongo

# EXTRACT

In [23]:
# *** Import CSV files ***

csv_file = "Resources/incidents.csv"
incidents_df = pd.read_csv(csv_file)
incidents_df.head()

,Name,Link to article,Year,Deaths,Injuries,Place,Place (State),Store / Location,Cause of Death / Injury
0,10 Injured in Southern California Black Friday...,http://abclocal.go.com/kabc/story?section=news...,2006,0,10,"Torrance, California",California,Del Amo Fashion Center,Stampede
1,"Salt Lake Tribune, The : Red-hot on Black Friday‎",http://nl.newsbank.com/nl-search/we/Archives?p...,2006,0,1,"Murray, Utah",Utah,Fashion Place Mall,Stampede
2,Worker dies at Long Island Wal-Mart after bein...,http://www.nydailynews.com/new-york/worker-die...,2008,1,4,"Long Island, New York",New York,Wal-Mart,Stampede
3,Southern California Toys 'R' Us Shooting Leave...,http://www.huffingtonpost.com/2008/11/28/socal...,2008,2,2,"Palm Desert, California",California,Toys 'R' Us,Shooting
4,Clarksville Woman Trampled During Black Friday...,http://www.newschannel5.com/global/story.asp?s...,2009,0,1,"Clarksville, Tennessee",Tennessee,Toys 'R' Us,Stampede


In [24]:
incidents_df = incidents_df[["Name", "Year", "Deaths", "Injuries", "Place (State)", "Cause of Death / Injury"]].copy()
incidents_df.head()

,Name,Year,Deaths,Injuries,Place (State),Cause of Death / Injury
0,10 Injured in Southern California Black Friday...,2006,0,10,California,Stampede
1,"Salt Lake Tribune, The : Red-hot on Black Friday‎",2006,0,1,Utah,Stampede
2,Worker dies at Long Island Wal-Mart after bein...,2008,1,4,New York,Stampede
3,Southern California Toys 'R' Us Shooting Leave...,2008,2,2,California,Shooting
4,Clarksville Woman Trampled During Black Friday...,2009,0,1,Tennessee,Stampede


In [25]:
csv_file = "Resources/places.csv"
places_df = pd.read_csv(csv_file)
places_df.head()

,Location,State,Deaths,Injuries
0,"Newport, Arkansas",Arkansas,0,1
1,"El Camino Real, California",California,0,1
2,"Los Angeles, California",California,0,20
3,"Palm Desert, California",California,2,2
4,"Palo Alto, California",California,2,5


In [26]:
csv_file = "Resources/BlackFriday.csv"
blackfriday_df = pd.read_csv(csv_file)
blackfriday_df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [27]:
# *** Screen Scrape tables ***

state_blackfriday_violence_url = "https://www.reviews.org/trends/high-risk-states-for-black-friday-violence/"

blackfriday_death_count_url = "http://blackfridaydeathcount.com"

In [28]:
state_blackfriday_violence_table = pd.read_html(state_blackfriday_violence_url, header=0)
state_df = state_blackfriday_violence_table[0]
state_df.head()

,Our ranking,State,Violent Crime ranking,Interest in Black Friday
0,1,ARKANSAS,6,15
1,2,TENNESSEE,3,14
2,3,WEST VIRGINIA,28,1
3,4,NORTH CAROLINA,24,21
4,5,ALABAMA,8,26


In [29]:
blackfriday_death_count_table = pd.read_html(blackfriday_death_count_url, header=0)
bf_death = blackfriday_death_count_table[0]
bf_death.head()

,Unnamed: 0,Unnamed: 1,Deaths,Injuries
0,2018,"Man, 27, shot to death in parking lot in South...",1,-
1,2018,Man wounded in shooting inside N.J. mall at th...,-,1
2,2018,Two people stabbed at Macy's in Destiny USA,-,2
3,2018,Glance at attractive woman sparked Memphis mal...,-,1
4,2018,One dead and 2 wounded— including 12-year-old ...,1,2


# TRANSFORM

In [30]:
cleaned_incidents_df = incidents_df.rename(columns={"Place (State)":"Place", "Cause of Death / Injury":"Cause_of_Death_or_Injury"})
cleaned_incidents_df.head()

,Name,Year,Deaths,Injuries,Place,Cause_of_Death_or_Injury
0,10 Injured in Southern California Black Friday...,2006,0,10,California,Stampede
1,"Salt Lake Tribune, The : Red-hot on Black Friday‎",2006,0,1,Utah,Stampede
2,Worker dies at Long Island Wal-Mart after bein...,2008,1,4,New York,Stampede
3,Southern California Toys 'R' Us Shooting Leave...,2008,2,2,California,Shooting
4,Clarksville Woman Trampled During Black Friday...,2009,0,1,Tennessee,Stampede


In [31]:
blackfriday_df = blackfriday_df[["User_ID", "Product_ID", "Gender", "Age", "Occupation", "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3", "Purchase"]].copy()
blackfriday_df.head()

,User_ID,Product_ID,Gender,Age,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,0,8,NaN,NaN,7969


In [32]:
new_state_df = state_df[['Our ranking','State','Violent Crime ranking']]
new_state_df2=new_state_df.rename(columns={"Our ranking":"ranking","State":"state","Violent Crime ranking":"vio_crime_rank"})
new_state_df2.head()

,ranking,state,vio_crime_rank
0,1,ARKANSAS,6
1,2,TENNESSEE,3
2,3,WEST VIRGINIA,28
3,4,NORTH CAROLINA,24
4,5,ALABAMA,8


In [34]:
bf_death_df = bf_death.rename(columns={"Unnamed: 0":"year","Unnamed: 1":"article_title","Deaths":"deaths","Injuries":"injuries"})

bf_death_df2 = bf_death_df.replace('-',0)

bf_death_df3 = bf_death_df2.dropna()

bf_death_df4 = bf_death_df3.reset_index(drop=True)
bf_death_df4.drop(bf_death_df2.index[47], inplace=True)
bf_death_df5 = bf_death_df4.reset_index(drop=True)
bf_death_df5.head(10)

,year,article_title,deaths,injuries
0,2018,"Man, 27, shot to death in parking lot in South...",1,0
1,2018,Man wounded in shooting inside N.J. mall at th...,0,1
2,2018,Two people stabbed at Macy's in Destiny USA,0,2
3,2018,Glance at attractive woman sparked Memphis mal...,0,1
4,2018,One dead and 2 wounded— including 12-year-old ...,1,2
5,2017,Black Friday altercation in Kmart leaves man w...,0,1
6,2017,Black Friday Missouri mall shooting leaves tee...,0,1
7,2017,"One male shot, one male stabbed at Willowbrook...",0,2
8,2017,"Brawls close Alabama shopping center, one trea...",0,1
9,2016,San Antonio man helped a woman being beaten in...,1,2


# LOAD

In [42]:
# *** Load Data to MySQL ***

rds_connection_string = "root:kateH1031@localhost:3306/blackfriday_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [43]:
cleaned_incidents_df.to_sql(name='incidents', con=engine, if_exists='append', index=False)

In [44]:
places_df.to_sql(name='places', con=engine, if_exists='append', index=False)

In [45]:
blackfriday_df.to_sql(name='blackfriday', con=engine, if_exists='append', index=False)

In [46]:
new_state_df2.to_sql(name='state_violentcrimes',con=engine,if_exists='append', index=False)

In [47]:
bf_death_df5.to_sql(name='incident_by_year',con=engine,if_exists='append', index=False)

In [48]:
pd.read_sql_query('select * from incidents',con=engine).head()

,Name,Year,Deaths,Injuries,Place,Cause_of_Death_or_Injury
0,10 Injured in Southern California Black Friday...,2006,0.0,10.0,California,Stampede
1,"Salt Lake Tribune, The : Red-hot on Black Friday‎",2006,0.0,1.0,Utah,Stampede
2,Worker dies at Long Island Wal-Mart after bein...,2008,1.0,4.0,New York,Stampede
3,Southern California Toys 'R' Us Shooting Leave...,2008,2.0,2.0,California,Shooting
4,Clarksville Woman Trampled During Black Friday...,2009,0.0,1.0,Tennessee,Stampede


In [50]:
pd.read_sql_query('select * from places',con=engine).head()

,Location,State,Deaths,Injuries
0,"Newport, Arkansas",Arkansas,0.0,1.0
1,"El Camino Real, California",California,0.0,1.0
2,"Los Angeles, California",California,0.0,20.0
3,"Palm Desert, California",California,2.0,2.0
4,"Palo Alto, California",California,2.0,5.0


In [51]:
pd.read_sql_query('select * from blackfriday',con=engine).head()

,User_ID,Product_ID,Gender,Age,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10.0,0.0,3,None,None,8370
1,1000001,P00248942,F,0-17,10.0,0.0,1,6,14,15200
2,1000001,P00087842,F,0-17,10.0,0.0,12,None,None,1422
3,1000001,P00085442,F,0-17,10.0,0.0,12,14,None,1057
4,1000002,P00285442,M,55+,16.0,0.0,8,None,None,7969


In [52]:
pd.read_sql_query('select * from state_violentcrimes',con=engine).head()

,ranking,state,vio_crime_rank
0,1,ARKANSAS,6
1,2,TENNESSEE,3
2,3,WEST VIRGINIA,28
3,4,NORTH CAROLINA,24
4,5,ALABAMA,8


In [53]:
pd.read_sql_query('select * from incident_by_year',con=engine).head()

,year,article_title,deaths,injuries
0,2018,"Man, 27, shot to death in parking lot in South...",1,0
1,2018,Man wounded in shooting inside N.J. mall at th...,0,1
2,2018,Two people stabbed at Macy's in Destiny USA,0,2
3,2018,Glance at attractive woman sparked Memphis mal...,0,1
4,2018,One dead and 2 wounded— including 12-year-old ...,1,2


In [54]:
# *** Load Data to MongoDB ***
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [55]:
state_bf_db = client.state_bf_db
collection_state_bf = state_bf_db.items

bf_dc_db = client.bf_deathcount_db
collection_bf_deathcount = bf_dc_db.items

In [ ]:
response1 = requests.get(state_blackfriday_violence_url)
soup1 = BeautifulSoup(response1.text, 'lxml')

response2 = requests.get(blackfriday_death_count_url)
soup2 = BeautifulSoup(response2.text, 'lxml')

In [ ]:
results = soup1.find_all('tr',class_="default-row")

for result in results:
    
    try:
        tds = result.find_all('td')
        our_ranking = tds[0].text
        
        state=tds[1].text
        
        violent_crime_ranking=tds[2].text
        
        interest_in_bf=tds[3].text
        
        if (our_ranking and state and violent_crime_ranking and interest_in_bf):
            print("------------------")
            print(our_ranking)
            print(state)
            print(violent_crime_ranking)
            print(interest_in_bf)
            
            post = {
                'rank':our_ranking,
                'state':state,
                'violent_crime_ranking':violent_crime_ranking,
                'interest_in_black_friday':interest_in_bf
            }
            
            collection_state_bf.insert_one(post)
            
    except Exception as e:
        print(e)

In [ ]:
# Display items in MongoDB collection
listings = state_bf_db.items.find()

for listing in listings:
    pprint(listing)

In [ ]:
bf_death_df6= bf_death_df5
bf_death_df6.head()

In [ ]:
bf_death_df6.insert(0,'New_ID', f'ind_')

In [ ]:
bf_death_df7 = bf_death_df6.set_index('New_ID')
bf_death_df7.head()

In [ ]:
# SECOND TABLE TO MONGO DB

bf_death_dict = bf_death_df7.to_dict('index')
bf_death_dict

In [ ]:
collection_bf_deathcount.insert(bf_death_dict)